In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [2]:
def simulate_gbm(M, N, T, S0, mu, sigma):
    np.random.seed(1234)
    dt = T/N
    dW = np.random.normal(0, np.sqrt(dt),size=(M,N)).T
    St = np.exp((mu-sigma ** 2 / 2) * dt + sigma * dW)
    St = np.vstack([np.ones(M),St])
    St[0] = S0
    St = St.cumprod(axis=0)
    return St


### in the money & discounted_price[k+1:][:,exercise] = 0 

In [105]:

# discounted price for all iterations - a list
Price = [0]*n
# price info from iteration 1 to current iteration - a dictionary with k as the key
X = dict()
Y = dict()
# model info for each time step - a dictionary
alphas = dict()

# initializaiton:
paths = simulate_gbm(int(N/n), M, T, S0, r, sigma)
euro_discounted_prices = np.ones_like(paths) * np.exp(-r/M)
final_payoff = strike - paths[-1]
euro_discounted_prices[0] = np.where(final_payoff > 0, final_payoff, 0)
euro_discounted_prices = euro_discounted_prices.cumprod(axis=0)
euro_discounted_prices = euro_discounted_prices[::-1]
for k in range(1,M+1):
    in_the_money = paths[k,:] < strike
    x1 = paths[k,in_the_money]
    x2 = x1*x1
    x3 = k*np.ones_like(x1)
    x4 = k*x1
    x5 = k*x2
    xs = np.column_stack([x1,x2,x3,x4,x5])
    X[k] = xs
    if k==M:
        Y[k] = euro_discounted_prices[k,in_the_money]
    else:
        Y[k]= euro_discounted_prices[k+1,in_the_money] * np.exp(-r/M)
    model_sklearn = LinearRegression()
    model = model_sklearn.fit(X[k], Y[k])
    alphas[k] = np.append(np.array([model.intercept_]),model.coef_)

# iterations
for i in range(1,n+1):
    # for one iteration
    # step 1: paths simulation
    paths =simulate_gbm(int(N/n), M, T, S0, r, sigma)
    
    # step 2.1: payoff value calculation
    payoff_values = strike - paths
    payoff_values[payoff_values<0] = 0

    # step 2.2: continuation value estimate:
    continuation_values = np.zeros_like(paths)
    for k in range(1,M+1):
        x0 = np.ones_like(paths[k]) # intercept
        x1 = paths[k] # S
        x2 = x1 * x1 # s^2
        x3 = k * np.ones_like(x1) # t
        x4 = k * x1 # tS
        x5 = k * x2 # tS^2
        xs = np.column_stack([x0, x1, x2, x3, x4, x5])
        continuation_values[k] = xs.dot(alphas[k])
    
    # step 3: comparison between continuation value and exercise value
    discounted_price = payoff_values
    for k in range(M-1,0,-1):
        no_exercise =  payoff_values[k] < continuation_values[k]
        exercise =  payoff_values[k] > continuation_values[k]
        discounted_price[k][no_exercise]= discounted_price[k+1][no_exercise] * np.exp(-r/M)
        discounted_price[k+1:][:,exercise] = 0 

    # step 4: sum of t=1 prices
    Price[i-1]= np.exp(-r/M) * discounted_price[1].sum()

    # step 5: update the regression models
    for k in range(1,M+1):
        in_the_money = paths[k,:] < strike
        x1 = paths[k,in_the_money]
        x2 = x1 * x1
        x3 = k * np.ones_like(x1)
        x4 = k * x1 
        x5 = k * x2
        xs = np.column_stack([x1, x2, x3, x4, x5])
        X[k]= np.vstack([X[k],xs])
        if k==M:
            Y[k] = np.hstack([Y[k],discounted_price[k, in_the_money]])
        else:
            Y[k] = np.hstack([Y[k],discounted_price[k+1, in_the_money] * np.exp(-r/M)])
        model_sklearn = LinearRegression()
        model = model_sklearn.fit(X[k], Y[k])
        alphas[k] = np.append(np.array([model.intercept_]),model.coef_)

# weights given to different iterations
W = np.array([0.99*(i-1) for i in range(1,n+1)])
W = np.tanh(W)
W = 1 - 0.5 * (1 - W)
Price = np.array(Price) / (N/n)
Option_price = sum(Price * W) / W.sum()
print(Price)




[4.15498492 4.00787149 3.97911279 3.96747823 3.96513219 3.96399499
 3.96252708 3.9609431  3.9598402  3.95921432]


###  with discounted_price[k+1:][:,exercise] = 0 

In [107]:

# discounted price for all iterations - a list
Price = [0]*n
# price info from iteration 1 to current iteration - a dictionary with k as the key
X = dict()
Y = dict()
# model info for each time step - a dictionary
alphas = dict()

# initializaiton:
paths = simulate_gbm(int(N/n), M, T, S0, r, sigma)
euro_discounted_prices = np.ones_like(paths) * np.exp(-r/M)
final_payoff = strike - paths[-1]
euro_discounted_prices[0] = np.where(final_payoff > 0, final_payoff, 0)
euro_discounted_prices = euro_discounted_prices.cumprod(axis=0)
euro_discounted_prices = euro_discounted_prices[::-1]
for k in range(1,M+1):

    x1 = paths[k]
    x2 = x1*x1
    x3 = k*np.ones_like(x1)
    x4 = k*x1
    x5 = k*x2
    xs = np.column_stack([x1,x2,x3,x4,x5])
    X[k] = xs
    if k==M:
        Y[k] = euro_discounted_prices[k]
    else:
        Y[k]= euro_discounted_prices[k+1] * np.exp(-r/M)
    model_sklearn = LinearRegression()
    model = model_sklearn.fit(X[k], Y[k])
    alphas[k] = np.append(np.array([model.intercept_]),model.coef_)

# iterations
for i in range(1,n+1):
    # for one iteration
    # step 1: paths simulation
    paths =simulate_gbm(int(N/n), M, T, S0, r, sigma)
    
    # step 2.1: payoff value calculation
    payoff_values = strike - paths
    payoff_values[payoff_values<0] = 0

    # step 2.2: continuation value estimate:
    continuation_values = np.zeros_like(paths)
    for k in range(1,M+1):
        x0 = np.ones_like(paths[k]) # intercept
        x1 = paths[k] # S
        x2 = x1 * x1 # s^2
        x3 = k * np.ones_like(x1) # t
        x4 = k * x1 # tS
        x5 = k * x2 # tS^2
        xs = np.column_stack([x0, x1, x2, x3, x4, x5])
        continuation_values[k] = xs.dot(alphas[k])
    
    # step 3: comparison between continuation value and exercise value
    discounted_price = payoff_values
    for k in range(M-1,0,-1):
        no_exercise =  payoff_values[k] < continuation_values[k]
        exercise =  payoff_values[k] > continuation_values[k]
        discounted_price[k][no_exercise]= discounted_price[k+1][no_exercise] * np.exp(-r/M)
        discounted_price[k+1:][:,exercise] = 0 

    # step 4: sum of t=1 prices
    Price[i-1]= np.exp(-r/M) * discounted_price[1].sum()

    # step 5: update the regression models
    for k in range(1,M+1):
 
        x1 = paths[k]
        x2 = x1 * x1
        x3 = k * np.ones_like(x1)
        x4 = k * x1 
        x5 = k * x2
        xs = np.column_stack([x1, x2, x3, x4, x5])
        X[k]= np.vstack([X[k],xs])
        if k==M:
            Y[k] = np.hstack([Y[k],discounted_price[k]])
        else:
            Y[k] = np.hstack([Y[k],discounted_price[k+1] * np.exp(-r/M)])
        model_sklearn = LinearRegression()
        model = model_sklearn.fit(X[k], Y[k])
        alphas[k] = np.append(np.array([model.intercept_]),model.coef_)

# weights given to different iterations
W = np.array([0.99*(i-1) for i in range(1,n+1)])
W = np.tanh(W)
W = 1 - 0.5 * (1 - W)
Price = np.array(Price) / (N/n)
Option_price = sum(Price * W) / W.sum()
print(Price)


[4.18658215 4.01018724 3.97968692 3.96766232 3.96516241 3.96402349
 3.96301391 3.96183608 3.95985847 3.95929916]


### in the money

In [108]:

# discounted price for all iterations - a list
Price = [0]*n
# price info from iteration 1 to current iteration - a dictionary with k as the key
X = dict()
Y = dict()
# model info for each time step - a dictionary
alphas = dict()

# initializaiton:
paths = simulate_gbm(int(N/n), M, T, S0, r, sigma)
euro_discounted_prices = np.ones_like(paths) * np.exp(-r/M)
final_payoff = strike - paths[-1]
euro_discounted_prices[0] = np.where(final_payoff > 0, final_payoff, 0)
euro_discounted_prices = euro_discounted_prices.cumprod(axis=0)
euro_discounted_prices = euro_discounted_prices[::-1]
for k in range(1,M+1):
    in_the_money = paths[k,:] < strike
    x1 = paths[k,in_the_money]
    x2 = x1*x1
    x3 = k*np.ones_like(x1)
    x4 = k*x1
    x5 = k*x2
    xs = np.column_stack([x1,x2,x3,x4,x5])
    X[k] = xs
    if k==M:
        Y[k] = euro_discounted_prices[k,in_the_money]
    else:
        Y[k]= euro_discounted_prices[k+1,in_the_money] * np.exp(-r/M)
    model_sklearn = LinearRegression()
    model = model_sklearn.fit(X[k], Y[k])
    alphas[k] = np.append(np.array([model.intercept_]),model.coef_)

# iterations
for i in range(1,n+1):
    # for one iteration
    # step 1: paths simulation
    paths =simulate_gbm(int(N/n), M, T, S0, r, sigma)
    
    # step 2.1: payoff value calculation
    payoff_values = strike - paths
    payoff_values[payoff_values<0] = 0

    # step 2.2: continuation value estimate:
    continuation_values = np.zeros_like(paths)
    for k in range(1,M+1):
        x0 = np.ones_like(paths[k]) # intercept
        x1 = paths[k] # S
        x2 = x1 * x1 # s^2
        x3 = k * np.ones_like(x1) # t
        x4 = k * x1 # tS
        x5 = k * x2 # tS^2
        xs = np.column_stack([x0, x1, x2, x3, x4, x5])
        continuation_values[k] = xs.dot(alphas[k])
    
    # step 3: comparison between continuation value and exercise value
    discounted_price = payoff_values
    for k in range(M-1,0,-1):
        no_exercise =  payoff_values[k] < continuation_values[k]
        discounted_price[k][no_exercise]= discounted_price[k+1][no_exercise] * np.exp(-r/M)
 

    # step 4: sum of t=1 prices
    Price[i-1]= np.exp(-r/M) * discounted_price[1].sum()

    # step 5: update the regression models
    for k in range(1,M+1):
        in_the_money = paths[k,:] < strike
        x1 = paths[k,in_the_money]
        x2 = x1 * x1
        x3 = k * np.ones_like(x1)
        x4 = k * x1 
        x5 = k * x2
        xs = np.column_stack([x1, x2, x3, x4, x5])
        X[k]= np.vstack([X[k],xs])
        if k==M:
            Y[k] = np.hstack([Y[k],discounted_price[k, in_the_money]])
        else:
            Y[k] = np.hstack([Y[k],discounted_price[k+1, in_the_money] * np.exp(-r/M)])
        model_sklearn = LinearRegression()
        model = model_sklearn.fit(X[k], Y[k])
        alphas[k] = np.append(np.array([model.intercept_]),model.coef_)

# weights given to different iterations
W = np.array([0.99*(i-1) for i in range(1,n+1)])
W = np.tanh(W)
W = 1 - 0.5 * (1 - W)
Price = np.array(Price) / (N/n)
Option_price = sum(Price * W) / W.sum()
print(Price)





[4.15498492 4.17411198 4.17861158 4.17489674 4.1824449  4.16934712
 4.17295604 4.17023433 4.16312849 4.16567687]


In [ ]:
# Parameters
# number of exercise dates per year
M = 50
# total number of paths
N = 1000000
# number of iterations
n = 100
# time in years
T = 1
# initial stock price
S0 = 36
# volatility
sigma = 0.2
# strike
strike = 40
# let the annual risk-free interest rate
r = 0.06


# discounted price for all iterations - a list
Price = [0]*n
# price info from iteration 1 to current iteration - a dictionary with k as the key
X = dict()
Y = dict()
# model info for each time step - a dictionary
alphas = dict()

# initializaiton:
paths = simulate_gbm(int(N/n), M, T, S0, r, sigma)
euro_discounted_prices = np.ones_like(paths) * np.exp(-r/M)
final_payoff = strike - paths[-1]
euro_discounted_prices[0] = np.where(final_payoff > 0, final_payoff, 0)
euro_discounted_prices = euro_discounted_prices.cumprod(axis=0)
euro_discounted_prices = euro_discounted_prices[::-1]
for k in range(1,M+1):
  
    x1 = paths[k]
    x2 = x1*x1
    x3 = k*np.ones_like(x1)
    x4 = k*x1
    x5 = k*x2
    xs = np.column_stack([x1,x2,x3,x4,x5])
    X[k] = xs
    if k==M:
        Y[k] = euro_discounted_prices[k]
    else:
        Y[k]= euro_discounted_prices[k+1] * np.exp(-r/M)
    model_sklearn = LinearRegression()
    model = model_sklearn.fit(X[k], Y[k])
    alphas[k] = np.append(np.array([model.intercept_]),model.coef_)

# iterations
for i in range(1,n+1):
    # for one iteration
    # step 1: paths simulation
    paths =simulate_gbm(int(N/n), M, T, S0, r, sigma)
    
    # step 2.1: payoff value calculation
    payoff_values = strike - paths
    payoff_values[payoff_values<0] = 0

    # step 2.2: continuation value estimate:
    continuation_values = np.zeros_like(paths)
    for k in range(1,M+1):
        x0 = np.ones_like(paths[k]) # intercept
        x1 = paths[k] # S
        x2 = x1 * x1 # s^2
        x3 = k * np.ones_like(x1) # t
        x4 = k * x1 # tS
        x5 = k * x2 # tS^2
        xs = np.column_stack([x0, x1, x2, x3, x4, x5])
        continuation_values[k] = xs.dot(alphas[k])
    
    # step 3: comparison between continuation value and exercise value
    discounted_price = payoff_values
    for k in range(M-1,0,-1):
        no_exercise =  payoff_values[k] < continuation_values[k]
        discounted_price[k][no_exercise]= discounted_price[k+1][no_exercise] * np.exp(-r/M)


    # step 4: sum of t=1 prices
    Price[i-1]= np.exp(-r/M) * discounted_price[1].sum()

    # step 5: update the regression models
    for k in range(1,M+1):

        x1 = paths[k]
        x2 = x1 * x1
        x3 = k * np.ones_like(x1)
        x4 = k * x1 
        x5 = k * x2
        xs = np.column_stack([x1, x2, x3, x4, x5])
        X[k]= np.vstack([X[k],xs])
        if k==M:
            Y[k] = np.hstack([Y[k],discounted_price[k]])
        else:
            Y[k] = np.hstack([Y[k],discounted_price[k+1] * np.exp(-r/M)])
        model_sklearn = LinearRegression()
        model = model_sklearn.fit(X[k], Y[k])
        alphas[k] = np.append(np.array([model.intercept_]),model.coef_)

# weights given to different iterations
W = np.array([0.99*(i-1) for i in range(1,n+1)])
W = np.tanh(W)
W = 1 - 0.5 * (1 - W)

# calculate final price
Price = np.array(Price) / (N/n)
Option_price = sum(Price * W) / W.sum()
print(Price)





### Testing on exercise boundary


In [5]:
def newtons_method(alpha, k, strike, initial_guess=35, tolerance=1e-7, max_iterations=1000):
    # Initial guess
    x_n = initial_guess
    
    for n in range(max_iterations):
        # Calculate f(x_n) using the given formula
        fx_n = (alpha[0] + alpha[3] * k - strike) + (alpha[1] + alpha[4] * k + 1) * x_n + (alpha[2] + alpha[5] * k) * x_n**2
        
        # Calculate f'(x_n) using the given derivative formula
        fpx_n = (alpha[1] + alpha[4] * k + 1) + 2 * (alpha[2] + alpha[5] * k) * x_n
        
        if abs(fpx_n) < tolerance:  # Prevent division by zero
            print("Derivative too small, no convergence.")
            return None
        
        # Newton's update
        x_n1 = x_n - fx_n / fpx_n
        
        # Check for convergence
        if abs(x_n1 - x_n) < tolerance:
            return x_n1
        
        x_n = x_n1
    
    # If no convergence, return None
    print("Max iterations exceeded, no convergence.")
    return None



In [6]:
# Parameters
# number of exercise dates per year
M = 50
# total number of paths
N = 100000
# number of iterations
n = 100
# time in years
T = 1
# initial stock price
S0 = 36
# volatility
sigma = 0.2
# strike
strike = 40
# let the annual risk-free interest rate
r = 0.06


# discounted price for all iterations - a list
Price = [0]*n
# price info from iteration 1 to current iteration - a dictionary with k as the key
X = dict()
Y = dict()
# model info for each time step - a dictionary
alphas = dict()

# weights for paths
w_i = np.array([])
w_UV = [1 - 2 * np.exp(- i / 1.2) for i in range(0,n+1)] 

# initializaiton:
paths = simulate_gbm(int(N/n), M, T, S0, r, sigma)
euro_discounted_prices = np.ones_like(paths) * np.exp(-r/M)
final_payoff = strike - paths[-1]
euro_discounted_prices[0] = np.where(final_payoff > 0, final_payoff, 0)
euro_discounted_prices = euro_discounted_prices.cumprod(axis=0)
euro_discounted_prices = euro_discounted_prices[::-1]

w_i = np.append(w_i, np.array([w_UV[0] ** (10 - 0)] * int(N/n)))

for k in range(1,M+1):
    x1 = paths[k]
    x2 = x1*x1
    x3 = k*np.ones_like(x1)
    x4 = k*x1
    x5 = k*x2
    xs = np.column_stack([x1,x2,x3,x4,x5])
    X[k] = xs
    if k==M:
        Y[k] = euro_discounted_prices[k]
    else:
        Y[k]= euro_discounted_prices[k+1] * np.exp(-r/M)
    
    
    model_sklearn = LinearRegression()
    model = model_sklearn.fit(X[k], Y[k])
    alphas[k] = np.append(np.array([model.intercept_]),model.coef_)

In [7]:
for i in range(1,n+1):
    # for one iteration
    # step 1: paths simulation
    paths =simulate_gbm(int(N/n), M, T, S0, r, sigma)
    
    # step 2.1: payoff value calculation
    payoff_values = strike - paths
    payoff_values[payoff_values<0] = 0

    # step 2.2: continuation value estimate:
    continuation_values = np.zeros_like(paths)
    for k in range(1,M+1):
        x0 = np.ones_like(paths[k]) # intercept
        x1 = paths[k] # S
        x2 = x1 * x1 # s^2
        x3 = k * np.ones_like(x1) # t
        x4 = k * x1 # tS
        x5 = k * x2 # tS^2
        xs = np.column_stack([x0, x1, x2, x3, x4, x5])
        continuation_values[k] = xs.dot(alphas[k])
    
    # step 3: comparison between continuation value and exercise value
    discounted_price = payoff_values
    for k in range(M-1,0,-1):
        
        no_exercise =  payoff_values[k] < continuation_values[k]
        discounted_price[k][no_exercise]= discounted_price[k+1][no_exercise] * np.exp(-r/M)
        
        


    # step 4: sum of t=1 prices
    Price[i-1]= np.exp(-r/M) * discounted_price[1].sum()

    # step 5: update the regression models
    w_i = np.append(w_i, np.array([w_UV[i] ** (10 - i)] * int(N/n)))
    for k in range(1,M+1):
        x1 = paths[k]
        x2 = x1 * x1
        x3 = k * np.ones_like(x1)
        x4 = k * x1 
        x5 = k * x2
        xs = np.column_stack([x1, x2, x3, x4, x5])
        X[k]= np.vstack([X[k],xs])
        if k==M:
            Y[k] = np.hstack([Y[k],discounted_price[k]])
        else:
            Y[k] = np.hstack([Y[k],discounted_price[k+1] * np.exp(-r/M)])

        root = newtons_method(alphas[k], k, strike)
        if root:
            exercise_boundary = root
            
        x = X[k][:,0] 
        beta_k = x.std()
        y_k = np.exp(- (x - exercise_boundary) ** 2 / (2 * beta_k ** 2))
        w_k = w_i * y_k
        w_k = np.maximum(w_k, 0)

        model_sklearn = LinearRegression()
        model = model_sklearn.fit(X[k], Y[k],sample_weight=w_k)
        alphas[k] = np.append(np.array([model.intercept_]),model.coef_)




Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations exceeded, no convergence.
Max iterations e

In [8]:
# weights given to different iterations
W = np.array([0.99*(i-1) for i in range(1,n+1)])
W = np.tanh(W)
W = 1 - 0.5 * (1 - W)

# calculate final price
Price = np.array(Price) / (N/n)
Option_price = sum(Price * W) / W.sum()
print(Price)
print(Option_price)

[4.12711863 4.12981563 4.12891122 4.13362512 4.16506897 4.21798715
 4.252183   4.26047655 4.28249658 4.31135469 4.33442816 4.33882888
 4.33815033 4.33856969 4.34105746 4.33749886 4.33865333 4.34778191
 4.34218443 4.3517226  4.34970479 4.36169999 4.36939098 4.38125736
 4.3753813  4.37672091 4.37755916 4.37707046 4.39218246 4.405748
 4.39192164 4.3810447  4.39168571 4.37269971 4.37939214 4.40104022
 4.36531238 4.37498733 4.35334999 4.38982186 4.38059736 4.40752409
 4.35317247 4.40743742 4.35494011 4.40251259 4.35625149 4.40803497
 4.34615859 4.40480779 4.35049681 4.39737772 4.34954725 4.39987101
 4.34824003 4.39781266 4.34532491 4.39918155 4.33834526 4.39492686
 4.33774553 4.39502779 4.33556845 4.39543392 4.32662638 4.39143456
 4.3295942  4.39621429 4.33561876 4.39792865 4.33548569 4.40064864
 4.33370697 4.40076018 4.33622569 4.40168646 4.33861882 4.4050409
 4.33770401 4.40524135 4.33032307 4.3983474  4.32591394 4.39775029
 4.32574211 4.39321669 4.32692258 4.40009195 4.32661861 4.3962451